In [7]:
import wbdata
import pandas as pd
import datetime

In [ ]:
#Colection of data from World Bank
data_date = (datetime.datetime(2010, 1, 1), datetime.datetime(2024, 12, 31))

indicators = {'NY.GDP.MKTP.CD' : 'GDP_USD'}

data = wbdata.get_dataframe(indicators, country='KEN')

print(data.head())


           GDP_USD
date              
2024  1.244987e+11
2023  1.080386e+11
2022  1.144490e+11
2021  1.097037e+11
2020  1.006575e+11


In [ ]:
#Setting the date to the required time and using the date format
start_date = datetime.datetime(2010, 1, 1)
end_date = datetime.datetime(2024, 1, 1)

gdp_data = data.reset_index()

gdp_data['date'] = pd.to_datetime(gdp_data['date'], format='%Y', errors='coerce')
gdp_data = gdp_data.sort_values('date')

gdp_data = gdp_data[(gdp_data['date'] >= start_date) & (gdp_data['date'] <= end_date)]

print(gdp_data.head())

         date       GDP_USD
14 2010-01-01  4.540562e+10
13 2011-01-01  4.686947e+10
12 2012-01-01  5.639670e+10
11 2013-01-01  6.167144e+10
10 2014-01-01  6.828580e+10


In [20]:
#Drop any missing values
gdp_new_data = gdp_data.dropna()

print(gdp_new_data.head())

         date       GDP_USD
14 2010-01-01  4.540562e+10
13 2011-01-01  4.686947e+10
12 2012-01-01  5.639670e+10
11 2013-01-01  6.167144e+10
10 2014-01-01  6.828580e+10


In [ ]:
#Reading the data from csv file
cbk_data = pd.read_csv("C:\DATA ANALYTICS\PYDATA\PyData_Project\data\Mobile_Payments_2010-2024.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HP ELITEBOOK 840 G6\AppData\Local\Temp\ipykernel_24968\3026982200.py:1: SyntaxWarning: invalid escape sequence '\D'
  cbk_data = pd.read_csv("C:\DATA ANALYTICS\PYDATA\PyData_Project\data\Mobile_Payments_2010-2024.csv")


In [27]:
cbk_data['date'] = pd.to_datetime(cbk_data['Year'].astype(str) + '-' + cbk_data['Month'], format='%Y-%B')
cbk_new_data = cbk_data.sort_values('date')
print(cbk_new_data.head())

     Year  Month  Active Agents  Active Agents.1  \
219  2007  March            307         0.020992   
218  2007  April            362         0.054944   
217  2007    May            447         0.107733   
216  2007   June            527         0.175652   
215  2007   July            681         0.268499   

     Total Agent Cash in Cash Out (Volume Million)  \
219                                       0.021714   
218                                       0.070000   
217                                       0.150000   
216                                       0.233661   
215                                       0.354298   

     Total Agent Cash in Cash Out (Value KSh billions)       date  
219                                           0.064391 2007-03-01  
218                                           0.220896 2007-04-01  
217                                           0.483709 2007-05-01  
216                                           0.720102 2007-06-01  
215                   

In [28]:
#Setting the date as the index
Mobile_data = cbk_new_data.set_index('date')
print(Mobile_data.head())

            Year  Month  Active Agents  Active Agents.1  \
date                                                      
2007-03-01  2007  March            307         0.020992   
2007-04-01  2007  April            362         0.054944   
2007-05-01  2007    May            447         0.107733   
2007-06-01  2007   June            527         0.175652   
2007-07-01  2007   July            681         0.268499   

            Total Agent Cash in Cash Out (Volume Million)  \
date                                                        
2007-03-01                                       0.021714   
2007-04-01                                       0.070000   
2007-05-01                                       0.150000   
2007-06-01                                       0.233661   
2007-07-01                                       0.354298   

            Total Agent Cash in Cash Out (Value KSh billions)  
date                                                           
2007-03-01                    